In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# загрузка и предобработка
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
# https://github.com/huggingface/transformers
# обучение, загрузка по формату библиотеки, оптимизаторы
# !pip install transformers
# !pip install datasets
# !pip install umap-learn
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
# обучение
import torch
from torch.utils.data import DataLoader
# предобработка и финальный отчет по метрикам
from sklearn import preprocessing
from sklearn.metrics import classification_report
# кластеризация
from sklearn.feature_extraction.text import CountVectorizer
import umap
from sklearn.cluster import DBSCAN
# прогресс бар
from tqdm import tqdm
# очистка кешей
import gc
# размер шрифта в графиках
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [ ]:
# df = pd.read_excel('C:/Users/Академик/PycharmProjects/News_parsing/Разметка/Разметка GigaChat_Sentiments_14.12.2023.xlsx')
# df.shape

In [ ]:
# df.head(3)

In [ ]:
# df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# df.label.value_counts()

In [ ]:
# df = df[['text', 'label']]

In [ ]:
# df_0 = df[df['label'] == 'Негативный'].sample(n=1632)

In [ ]:
# df_1 = df[df['label'] == 'Позитивный']

In [ ]:
# df = pd.concat((df_0, df_1), ignore_index=True)

In [ ]:
# df.label.value_counts()

In [ ]:
# # меняем названия колонок под huggin face
# df.columns = ['description', 'labels']
# df

In [ ]:
# from sklearn import preprocessing
# Label_encoder = preprocessing.LabelEncoder()
# Label_encoder.fit(df['labels'])
# Label_encoder.classes_

In [ ]:
# df['labels'] = Label_encoder.fit_transform(df['labels'])

In [ ]:
# df.labels.value_counts()

In [ ]:
# сохраняем
# df.to_csv('df_updated.csv', index=False)

In [ ]:
# np.save('classes.npy', Label_encoder.classes_)

In [ ]:
# # перемешиваем и рабиваем на train test
# df = df.sample(frac=1).reset_index(drop=True)
# train = df[:int(len(df)*0.8)]
# test =  df[int(len(df)*0.8):]

In [ ]:
# train.labels.value_counts(), test.labels.value_counts()

In [ ]:
# train.to_csv('train.csv', index=False)
# test.to_csv('test.csv', index=False)

In [ ]:
model_name = "ai-forever/ruBert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# если есть желание дообучить лучшую модель - оставить эту строчку кода, иначе убрать
# model_name = 'sber-80(-84)'
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           num_labels=2, 
                                                           output_attentions=False,
                                                           output_hidden_states=False,
                                                           ignore_mismatched_sizes=True)

In [ ]:
names = list(model.named_parameters())
names[:6]

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

In [ ]:
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=300, padding='max_length'), batched=True)

In [ ]:
# это стандартные колонки для формата пайторча, все кроме них убираем
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  #
# убираем их из загрузчика
dataset = dataset.remove_columns(list(set(list(dataset['train'].features.keys())) - set(pytorch_style_columns)))

In [ ]:
dataset

In [ ]:
# переврдим в формат пайторча
# сразу грузим на gpu, если есть cuda, иначе девайс стоит убрать из аргумента
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda')

In [ ]:
# установить 8 или больше, если более 16 гб видеопамяти
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=1)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=1)

In [ ]:
len(train_dataloader)

In [ ]:
# оптимизатор AdamW, лр стоит поперебирать
optimizer = AdamW(model.parameters(), lr=2e-6)
# количество эпох можно побольше поставить
num_epochs = 4
# количество шагов
num_training_steps = num_epochs * len(train_dataloader)

# будем линейно увеличивать первые 200 шагов
lr_scheduler = get_scheduler(
    "constant_with_warmup",
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

In [ ]:
next(iter(train_dataloader))

In [ ]:
%%time
# лучший f1, по нему будем сохранять
best_f1 = 0.
# будем отображать каждые 10% эпохи
show_train_loss_every_num_epoch = 0.1

# проходимся по всем эпохам
for epoch in range(num_epochs):
    # отображаем номера эпох
    print(40*'-', '\nepoch', epoch+1)
    # переводим в режим тренировки
    model.train()
    # смотрим на средний лосс за 10% эпохи
    losses = []
    # итерируемся по треин части
    for i, batch in enumerate(train_dataloader):

        # переводим в режим тренировки
        model.train()
        # print(batch)
        # переносим батч на гпу, где и модель
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        # получаем прогнозы
        outputs = model(**batch)
        # print(outputs)
        # получаем лосс встроенный вместе с моделью (кросс-энтропия)
        loss = outputs.loss
        # делаем обратный проход
        loss.backward()
        # шаг по градиенту
        optimizer.step()
        # шаг по скорости
        lr_scheduler.step()
        # шаг по оптимизатору
        optimizer.zero_grad()
        # фиксируем потери на треин
        losses.append(loss.item())
        # отображаем каждый 10% эпохи
        if i%int(len(train_dataloader)*show_train_loss_every_num_epoch)==int(len(train_dataloader)*show_train_loss_every_num_epoch)-1:
            print(f'train loss [{i*100/len(train_dataloader):.2f}%]: {np.array(losses).mean():.3f}')
            losses = []
            # валидируемся в конце эпохи
            print('\nvalidating')
            # загружаем все основные метрики
            f1 = load_metric('f1')
            acc = load_metric('accuracy')
            precision = load_metric('precision')
            recall = load_metric('recall')
            with torch.no_grad():
                # переводим в режим валидации
                model.eval()
                # проходимся по всем батчам из теста
                for batch in tqdm(test_dataloader):
                    # переносим их на гпу
                    batch = {k: v.to(device) for k, v in batch.items()}
                    # не обновляя параметры получаем прогнозы
                    with torch.no_grad():
                        outputs = model(**batch)
                    # получаем сразу логиты
                    logits = outputs.logits
                    # находим верный ответ
                    predictions = torch.argmax(logits, dim=-1)
                    # логируем в метрики по f1
                    f1.add_batch(predictions=predictions, references=batch["labels"])
                    acc.add_batch(predictions=predictions, references=batch["labels"])
                    precision.add_batch(predictions=predictions, references=batch["labels"])
                    recall.add_batch(predictions=predictions, references=batch["labels"])
                # находим взвешенные по кол-ву примеров на тест метрики
                print('weighted summary:')
                print('Test acc:', acc.compute()['accuracy'])
                print('Test precision:', precision.compute(average = 'weighted')['precision'])
                print('Test recall:', recall.compute(average = 'weighted')['recall'])
                f1_weighted = f1.compute(average = 'weighted')['f1']
                print('Test f1:', f1_weighted, '\n')
                # если текущая f1 лучше максимальной
                if f1_weighted > best_f1:
                    # максмальная становится текущей
                    best_f1 = f1_weighted
                    # сохраняем модель
                    model.save_pretrained(f"ruBert-base_f1_max={round(best_f1, 3)}")

    # переводим обратно в режим тренировки для следующей эпохи
    model.train()

In [ ]:
# np.save('classes.npy', Label_encoder.classes_)
Label_encoder = preprocessing.LabelEncoder()

# для того, чтобы восстановить кодировку целевых признаков как при обучении
Label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)

# валидируем по тесту
# реальные ответы и предсказания
true = []
preds = []

# переводим модель в режим инференса
model.eval()
# проходимся по батчам теста
for batch in tqdm(test_dataloader):
    # переносим батч на GPU
    batch = {k: v.to(device) for k, v in batch.items()}
    # без обновления параметров находим прогнозы
    with torch.no_grad():
        outputs = model(**batch)
    # получаем логиты
    logits = outputs.logits
    # из логитов прогноз
    predictions = torch.argmax(logits, dim=-1) #predictions = torch.argmax(logits, dim=-1)
#     print(round(float(predictions[0][1]),3))
    # добавляем пачки ответов и прогнозов в массивы
#     true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()
#     preds.append(float(predictions[0][1]))

# ковертируем ответы и прогнозы обратно в человеко-читаемые названия классов, смотрим report по всем метрикам по каждому классу
# print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

In [ ]:
# preds_df = pd.DataFrame(preds)
# preds_df.to_excel('Тестирование на 3 класса.xlsx')

In [ ]:
from sklearn.metrics import (precision_score, recall_score, f1_score, classification_report, 
                             accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score)

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
fact = list(test.labels)

In [ ]:
cm = confusion_matrix(fact, preds)
cm_display = ConfusionMatrixDisplay(cm).plot(cmap='Blues')

f1 = f1_score(fact, preds, average='macro')
print(f'f1_score = {round(f1, 2)}')

In [ ]:
# Если нужно изменить порог принятия решения
# final_pred = []
# for prob in preds:
#     if prob > 0.1:
#         answer = 1
#     else:
#         answer = 0
#     final_pred.append(answer)
# cm = confusion_matrix(fact, final_pred)
# cm_display = ConfusionMatrixDisplay(cm).plot(cmap='Blues')

In [ ]:
# Если нужно изменить порог принятия решения
# test_different = []
# for prob in preds:
#     if prob < 0.3:
#         test_different.append('Негативный')
#     elif prob >0.3 and prob < 0.7:
#         test_different.append('Нейтральный')
#     else:
#         test_different.append('Позитивный')
# pd.DataFrame(test_different).value_counts()